<a href="https://colab.research.google.com/github/tsnow2010/DATA601/blob/main/DATA601_Project1/src/Inflation_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import plotly.express as px


df = pd.read_csv('https://raw.githubusercontent.com/tsnow2010/DATA601/refs/heads/main/DATA601_Project1/data/movie_data.csv')


# Step 1: Drop all movies that have NaN in GDBOS column.
df_GDBOS = df.dropna(subset=['Gross Domestic Box Office Sales ($)'])
print(df.shape)
print(df_GDBOS.shape)


print(f'A total of {df.shape[0]-df_GDBOS.shape[0]} movies were removed.')

# Which years lost all of these movies?

(12397, 13)
(8074, 13)
A total of 4323 movies were removed.


In [55]:
# Removes '$' and ',' char's from the GDBOS column values and turns them into <<float>>.
df_GDBOS['Gross Domestic Box Office Sales ($)'] = df['Gross Domestic Box Office Sales ($)'].astype(str).apply(lambda x: ''.join([char for char in x if char not in ['$',',']])).astype(float)
df_GDBOS.sample(1)

<ipython-input-55-a931ca90addd>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Title,Year,Genre(s),IMDB,Rotten Tomatoes,Metacritic,TMDB,# of IMDB Votes,# of Awards,Gross Domestic Box Office Sales ($),Media,Directors,Origin
9773,Bad Santa 2,2016.0,"Comedy, Crime, Drama",5.6,23,38.0,5.4,"28,814",1 win & 2 nominations,17782178.0,movie,Mark Waters,"United States, United Kingdom, Canada"


In [56]:
# Group movies by year and sum the GDBOS for each movie in that year.
df_GDBOS_by_year = df_GDBOS.groupby('Year')['Gross Domestic Box Office Sales ($)'].sum().reset_index()
df_GDBOS_by_year.sample(1)


,Year,Gross Domestic Box Office Sales ($)
29,2001.0,7.945312e+09


In [60]:
# Define function to convert the GDBOS for a year and calculate the USD value in year 2023.
ANNUAL_INFLATION_RATE = .04
LAST_YEAR = 2023

def get_today_GDBOS(row):
  release_year = row['Year']
  years_elapsed = LAST_YEAR - release_year
  release_year_GDBOS = row['Gross Domestic Box Office Sales ($)']

  # Returns USD value in 2023 using the formula T = P[(1+r)**n - 1] where:
  # T = USD value in 2023
  # P = USD value at year of movie release
  # r = average annual inflation rate
  # n = number of years elapsed
  return release_year_GDBOS * (((1 + ANNUAL_INFLATION_RATE) ** years_elapsed)-1)

# Apply function to calculate USD value (in 2023) for total GDBOS by year
df_GDBOS_by_year_with_inflation = df_GDBOS_by_year.copy()
df_GDBOS_by_year_with_inflation['Gross Domestic Box Office Sales ($)'] = df_GDBOS_by_year.apply(get_today_GDBOS, 1)
df_GDBOS_by_year_with_inflation.rename(columns={'Gross Domestic Box Office Sales ($)': 'Total GDBOS ($ in 2023)'}, inplace=True)
df_GDBOS_by_year_with_inflation.sample(5)

,Year,Total GDBOS ($ in 2023)
49,2021.0,3.846181e+08
12,1984.0,1.096225e+10
1,1973.0,5.180575e+09
15,1987.0,9.793521e+09
38,2010.0,6.589068e+09


In [63]:
# Create bar graph displaying the total GDBOS by year, adjusted for inflation through year 2023.
fig = px.bar(
    df_GDBOS_by_year_with_inflation,
    x = 'Year',
    y = 'Total GDBOS ($ in 2023)',
    title = 'Gross Domestic Box Office Sales by Year, Adjusted for Inflation through 2023'
)
fig.show()

# Add annotations showing why lack of data from 2014 on.
# Finding which movies were removed from dataset may explain this.
# Also, keep in mind streaming services became very popular around 2018, and these movies do not have GDBOS.

1576.2500000000014
